In [1]:
# qiskit circuit & machine
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import qiskit_aer
from qiskit_ibm_runtime.fake_provider import FakeVigo

# widget-related
import anywidget
import traitlets

# host setting for dev mode
_SS_ = "http://localhost:5175/"

In [2]:
class CircuitWriter(anywidget.AnyWidget):
    _esm = _SS_ + "index.js?anywidget"
    circ_code = traitlets.Unicode().tag(sync=True)
    mode = traitlets.Unicode("write").tag(sync=True)

def getCircuitWriter():
    circuit_code = ""
    return CircuitWriter(circ_code=circuit_code)

In [3]:
w = getCircuitWriter()

In [4]:
w

CircuitWriter()

In [5]:
w.circ_code

' >> >>'